In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp

In [ ]:
H = lambda x: np.heaviside(x, 1)
sigmoid = lambda x: 1/(1 + np.exp(-x))

t0=0
tf=10
n=100

t = np.linspace(t0, tf, n)

X = (H(t) - H(t-2))*t - (H(t-2) - H(t-6))*(t-6)*0.5
x = interp1d(t, X)


# Ideal integrator: y' = u
f = lambda t, y: x(t)
y0 = np.array([0])

# Forward neuron: y -> u
K = 10
f = lambda t, y: (x(t) - y) * K
y0 = np.array([0])

# Forward neuron with activation point:
K = 100
slope = 100
Iup = 1.5
a = lambda u: sigmoid(slope*(x - Iup)) * u
f = lambda t, y: (a(x(t)) - y) * K
y0 = np.array([0])

# Bistable neuron:
K = 10
slope = 20
Iup = 1.5
Idown = 0.5
above = lambda x: sigmoid(slope*(x - Iup))
below = lambda x: 1-sigmoid(slope*(x - Idown))
# Two terms in the derivative where the first one pulls the output up when above the activation threshold,
# and the other pulls it down when below. When between the thresholds, the output more or less remains unchanged.
f = lambda t, y: (above(x(t))*(1 - y) - below(x(t))*y) * K
y0 = np.array([0])

solution = solve_ivp(f, (t0, tf), y0, method = "Radau") # Radau because equation is stiff
y = interp1d(solution.t, solution.y[0])
print(solution.t.size)

fig = px.line(x=t, y=[x(t), y(t), above(x(t)), below(x(t))], labels={"x": "t", "value": "activity"})
fig.data[0].name = "x(t)"
fig.data[1].name = "y(t)"
fig.add_trace(go.Scatter(x=solution.t, y=solution.y[0], mode="markers", name="solution points"))
fig.show()

In [ ]:
t0=0
tf=3
n=100

t = np.linspace(t0, tf, n)

X = H(t-1) - H(t-1.5) #0.5*((H(t) - H(t-2))*t - (H(t-2) - H(t-6))*(t-6)*0.5)
x = interp1d(t, X)

N = 1
K = 30
tau = K #10
slope = 20
gain = 1.1
above = lambda x, Iup: sigmoid(slope*(x - Iup))
below = lambda x, Idown: 1-sigmoid(slope*(x - Idown))
# Two terms in the derivative where the first one pulls the output up when above the activation threshold,
# and the other pulls it down when below. When between the thresholds, the output more or less remains unchanged.
f = lambda t, y: np.array([
    (above(x(t) + y[0]/N*gain, 0.6)*(1 - y[0]) - below(x(t) + y[0]/N*gain, 0.05)*y[0]) * K, # + tau*(y[0]/N*gain - y[0]),
    #tau*(y[0]/N*gain - y[1]),
])

initial = np.array([0, 0])
solution = solve_ivp(f, (t0, tf), initial, method = "Radau") # Radau because equation is stiff

n = interp1d(solution.t, solution.y[1]/N)
y0 = interp1d(solution.t, solution.y[0])

fig = px.line(x=t, y=[x(t), n(t), y0(t)], labels={"x": "t", "value": "activity"})
fig.data[0].name = "x(t)"
fig.data[1].name = "n(t)"
fig.data[2].name = "y_0(t)"
fig.show()

In [ ]:
t0=0
tf=10
n=100

t = np.linspace(t0, tf, n)

X = 0.1*((H(t) - H(t-2))*t + 1*(H(t-2) - H(t-3)))
#X = H(t-1)
x = interp1d(t, X)

N = 200
K = 20
slope = 500
gain = 0.98

above = lambda x, Iup: sigmoid(slope*(x - Iup))
below = lambda x, Idown: 1-sigmoid(slope*(x - Idown))

Iup = np.linspace(1/(N), 1.0, N)
#print(Iup)
#print(Iup[0])
#print(Iup[1] - Iup[0])
#print(Iup[2] - Iup[1])
deltaI = 10.0 / N # ensure overlap

print(deltaI)

f = lambda t, z: np.array([x(t)])
solution = solve_ivp(f, (t0, tf), np.array([0]), t_eval=t, method="Radau")
z = interp1d(solution.t, solution.y[0])

def f(t, y):
    # Input is recurrent connections plus network input
    I = np.sum(y)/N*gain + x(t)
    return (above(I, Iup)*(1-y) - below(I, Iup-deltaI)*y) * K
    
initial = np.zeros(N)
solution = solve_ivp(f, (t0, tf), initial) # Radau because equation is stiff

n = interp1d(solution.t, np.sum(solution.y, 0)/N)

fig = px.line(x=t, y=[x(t), z(t), n(t)], labels={"x": "t", "value": "activity"})
fig.data[0].name = "x(t)"
fig.data[1].name = "true integral"
fig.data[2].name = "n(t)"

#for i in range(0, N):
#    fig.add_trace(go.Scatter(x=solution.t, y=solution.y[i], name=f"y_{i}(t)"))

fig.show()